<a href="https://colab.research.google.com/github/greardhito/earthquake-cluster/blob/main/Project_Clustering_Earthquake_with_K_Medoids_and_K_Means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Install lib scikit-learn-extra

In [ ]:
!pip install scikit-learn-extra

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import glob
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import silhouette_score
import plotly.express as px

Gabungkan 12 data file menjadi 1 dataframe yang akan digunakan sebagai datasets

In [ ]:
# Terdapat 12 file data yang dibagi berdasarkan bulan

folder_path = "/content/drive/MyDrive/datasets"

# Membaca semua file CSV dari folder
all_files = glob.glob(folder_path + "/*.csv")

# List untuk menyimpan DataFrame dari setiap file CSV
df_list = []

# Loop untuk membaca dan mengecek setiap file CSV
for file in all_files:
    try:
        print(f"Membaca file: {file}")
        df_temp = pd.read_csv(file)  # Skip baris yang bermasalah

        # Append ke df_list jika tidak ada masalah
        df_list.append(df_temp)
    except Exception as e:
        print(f"Error saat membaca file {file}: {e}")

# Gabungkan semua DataFrame hanya jika tidak ada masalah
try:
    data = pd.concat(df_list, ignore_index=True, join='inner')
    print("Penggabungan berhasil!")
    print(data.info())
except ValueError as e:
    print(f"Error saat menggabungkan: {e}")


Membaca file: /content/drive/MyDrive/datasets/EQ Repository   Requested Data (Januari).csv
Membaca file: /content/drive/MyDrive/datasets/EQ Repository   Requested Data (Februari).csv
Membaca file: /content/drive/MyDrive/datasets/EQ Repository   Requested Data (Maret).csv
Membaca file: /content/drive/MyDrive/datasets/EQ Repository   Requested Data (April).csv
Membaca file: /content/drive/MyDrive/datasets/EQ Repository   Requested Data (Mei).csv
Membaca file: /content/drive/MyDrive/datasets/EQ Repository   Requested Data (Juni).csv
Membaca file: /content/drive/MyDrive/datasets/EQ Repository   Requested Data (Juli).csv
Membaca file: /content/drive/MyDrive/datasets/EQ Repository   Requested Data (Agustus).csv
Membaca file: /content/drive/MyDrive/datasets/EQ Repository   Requested Data (September).csv
Membaca file: /content/drive/MyDrive/datasets/EQ Repository   Requested Data (Oktober).csv
Membaca file: /content/drive/MyDrive/datasets/EQ Repository   Requested Data (November).csv
Membaca f

Step 1 Preprocessing Data


In [ ]:
# Step 1: Preprocessing Data

# Fitur yang akan digunakan
data = data[data['Magnitude'] > 4.0]
features = ['Latitude', 'Longitude', 'Magnitude']
X = data[features].dropna()  # Drop missing values
print(X)

       Latitude   Longitude  Magnitude
2     -0.718176  131.811783   4.113108
9      1.701497  127.046013   4.128382
21    -6.744121  130.099197   4.228891
23     2.861820  127.201645   4.119722
24     3.105412  127.173836   4.067209
...         ...         ...        ...
10303 -3.564852  101.975670   4.842660
10315 -5.974566  105.729675   4.707775
10319 -7.547539  128.575104   4.391770
10331 -2.611577  129.506699   4.773047
10335 -2.603221  129.547730   4.506215

[1806 rows x 3 columns]


Step 2


Menentukan jumlah cluster k terbaik menggunakan elbow method

Step 3

Clustering menggunakan K-means dan K-medoids

In [ ]:
k_range = range(4, 10)

# Initialize empty dictionaries to store Silhouette scores for each algorithm and K value
silhouette_scores_kmeans = {}
silhouette_scores_kmedoids = {}

# Loop through the defined K range
for k in k_range:

  kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
  kmeans_labels = kmeans.fit_predict(X)

  kmedoids = KMedoids(n_clusters=k, random_state=42)
  kmedoids_labels = kmedoids.fit_predict(X)

  # Calculate Silhouette score for K-Means
  silhouette_avg_kmeans = silhouette_score(X, kmeans_labels)
  silhouette_scores_kmeans[k] = silhouette_avg_kmeans

  # Calculate Silhouette score for K-Medoids
  silhouette_avg_kmedoids = silhouette_score(X, kmedoids_labels)
  silhouette_scores_kmedoids[k] = silhouette_avg_kmedoids

# Find the K with the highest Silhouette score for each algorithm
k_best_kmeans = max(silhouette_scores_kmeans, key=silhouette_scores_kmeans.get)
k_best_kmedoids = max(silhouette_scores_kmedoids, key=silhouette_scores_kmedoids.get)

# Print the best K values and their corresponding Silhouette scores
print(f"Best K for K-Means: {k_best_kmeans} (Silhouette score: {silhouette_scores_kmeans[k_best_kmeans]:.3f})")
print(f"Best K for K-Medoids: {k_best_kmedoids} (Silhouette score: {silhouette_scores_kmedoids[k_best_kmedoids]:.3f})")

Best K for K-Means: 6 (Silhouette score: 0.589)
Best K for K-Medoids: 4 (Silhouette score: 0.561)


In [ ]:
print("Silhouette Scores for K-Means:")
for k, score in silhouette_scores_kmeans.items():
    print(f"k = {k}: {score:.3f}")

print("\nSilhouette Scores for K-Medoids:")
for k, score in silhouette_scores_kmedoids.items():
    print(f"k = {k}: {score:.3f}")

Silhouette Scores for K-Means:
k = 4: 0.564
k = 5: 0.558
k = 6: 0.589
k = 7: 0.563
k = 8: 0.544
k = 9: 0.550

Silhouette Scores for K-Medoids:
k = 4: 0.561
k = 5: 0.505
k = 6: 0.553
k = 7: 0.512
k = 8: 0.501
k = 9: 0.485


In [ ]:
kmeans = KMeans(n_clusters=6, init='k-means++', random_state=42)
kmeans_labels = kmeans.fit_predict(X)

kmedoids = KMedoids(n_clusters=4, random_state=42)
kmedoids_labels = kmedoids.fit_predict(X)

X['kmeans_cluster'] = kmeans_labels + 1
X['kmedoids_cluster'] = kmedoids_labels + 1

In [ ]:
custom_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b',
                '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#aec7e8', '#ffbb78']

# Visualisasi Peta untuk K-Means
fig_kmeans = px.scatter_mapbox(
    X,
    lat="Latitude",
    lon="Longitude",
    color="kmeans_cluster",
    color_continuous_scale=custom_colors,
    size="Magnitude",
    size_max=5,
    zoom=3.5,
    center={"lat": -2.5489, "lon": 118.0153},
    mapbox_style="open-street-map"
)

fig_kmeans.update_layout(
    title="Peta Cluster K-Means",
    legend_title_text="Cluster"
)

fig_kmeans.show()

# Visualisasi Peta untuk K-Medoids
fig_kmedoids = px.scatter_mapbox(
    X,
    lat="Latitude",
    lon="Longitude",
    color="kmedoids_cluster",
    color_continuous_scale=custom_colors,
    size="Magnitude",
    size_max=5,
    zoom=3.5,
    center={"lat": -2.5489, "lon": 118.0153},
    mapbox_style="open-street-map"
)

fig_kmedoids.update_layout(
    title="Peta Cluster K-Medoids",
    legend_title_text="Cluster"
)

fig_kmedoids.show()